In [1]:
from model_wrappers.gpt2_wrapper import GPT2Wrapper
MODEL_NAME = "gpt2-medium"
# tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
wrapper = GPT2Wrapper(model_name = "gpt2-medium", use_cuda = True)

/usr/local/lib/python3.10/dist-packages/transformers/models/gpt2/modeling_gpt2.py:972: FutureWarning: `GPT2LMHeadModel.parallelize` is deprecated and will be removed in v5 of Transformers, you should load your model with `device_map='balanced'` in the call to `from_pretrained`. You can also provide your own `device_map` but it needs to be a dictionary module_name to device, so for instance {'transformer.h.0': 0, 'transformer.h.1': 1, ...}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/gpt2/modeling_gpt2.py:693: FutureWarning: `GPT2Model.parallelize` is deprecated and will be removed in v5 of Transformers, you should load your model with `device_map='balanced'` in the call to `from_pretrained`. You can also provide your own `device_map` but it needs to be a dictionary module_name to device, so for instance {'h.0': 0, 'h.1': 1, ...}
  warnings.warn(


In [6]:
# !ls
male_file = open(f"bias_evaluation/unqover/word_lists/subjects/male.txt", "rb")
female_file = open(f"bias_evaluation/unqover/word_lists/subjects/female.txt", "rb")
ethnicity_file = open(f"bias_evaluation/unqover/word_lists/subjects/ethnicity.txt", "rb")
religion_file = open(f"bias_evaluation/unqover/word_lists/subjects/religion.txt", "rb")

male_names = set()
female_names = set()
countries = set()
ethnicities = set()
religions = set()

for line in male_file:
  male_names.add(line.decode("utf-8")[14:-1].lower())

for line in female_file:
  female_names.add(line.decode("utf-8")[17:-1].lower())

for line in ethnicity_file:
  ethnicities.add(line.decode("utf-8")[7:-1].lower())

for line in religion_file:
  religions.add(line.decode("utf-8")[7:-1].lower())

with open(f"bias_evaluation/unqover/word_lists/subjects/country.txt", "rb") as country_file:
  country_lines = country_file.read().splitlines(True)
  for country in country_lines[6:74]:
    countries.add(country.decode("utf-8")[7:-1].lower())
  for demonym in country_lines[75:]:
    countries.add(demonym.decode("utf-8")[11:-1].lower())

In [ ]:
layer_idx_dict = {}
for layer in range(24):
    print(f"Scanning layer {layer}")
    for index in range(4096):
        if(index % 1000 == 0):
            print(f"Scanning index {index}")
        top_k = set([word.lower() for word, prob in wrapper.project_value_to_vocab(layer, index, 10)])
        if(len(top_k.intersection(set(countries))) >= 1):
            print(f"Layer: {layer}, Index: {index}")
            print(top_k)
            key = str(layer)+"_"+str(index)
            layer_idx_dict[key] = top_k

Scanning layer 0
Scanning index 0
Layer: 0, Index: 287
{'fn', ' fields', 'tex', 'م', ' britt', ' manners', 'syria', ' wag', 'grid', ' pastor'}
Layer: 0, Index: 434
{'australia', 'prin', 'married', 'amber', ' thin', 'grass', 'gently', 'abee', 'edge', ' candle'}
Scanning index 1000
Layer: 0, Index: 1402
{'izing', 'french', 'gey', 'zag', 'ize', 'izes', 'ene', 'ized', 'ization', 'viol'}
Scanning index 2000
Layer: 0, Index: 2353
{'glers', 'direction', ' detect', ' revived', 'resent', 'usable', 'lv', ' bought', 'chinese', 'toggle'}
Layer: 0, Index: 2672
{'rir', ' liberation', 'bbc', 'eq', 'alist', 'sah', 'cu', 'northern', 'houston', 'chinese'}
Scanning index 3000
Layer: 0, Index: 3621
{' antid', ' transc', ' eur', ' aloud', 'esco', 'italy', ' adin', ' showc', 'afort', ' italy'}
Layer: 0, Index: 3637
{'owa', 'mexico', 'cha', 'eting', 'etics', '�', 'ets', 'stals', 'ダ', 'edited'}
Scanning index 4000
Scanning layer 1
Scanning index 0
Layer: 1, Index: 1
{'atta', ' trave', 'igate', 'ogle', 'frames

In [ ]:
import pickle
temp_file = open("countries_layer_idx_dict.pkl", "wb")
pickle.dump(layer_idx_dict, temp_file)

In [ ]:
temp_file = open("countries_layer_idx_dict.pkl", "rb")

pickle.load(temp_file)


{'0_287': {' britt',
  ' fields',
  ' manners',
  ' pastor',
  ' wag',
  'fn',
  'grid',
  'syria',
  'tex',
  'م'},
 '0_434': {' candle',
  ' thin',
  'abee',
  'amber',
  'australia',
  'edge',
  'gently',
  'grass',
  'married',
  'prin'},
 '0_1402': {'ene',
  'french',
  'gey',
  'ization',
  'ize',
  'ized',
  'izes',
  'izing',
  'viol',
  'zag'},
 '0_2353': {' bought',
  ' detect',
  ' revived',
  'chinese',
  'direction',
  'glers',
  'lv',
  'resent',
  'toggle',
  'usable'},
 '0_2672': {' liberation',
  'alist',
  'bbc',
  'chinese',
  'cu',
  'eq',
  'houston',
  'northern',
  'rir',
  'sah'},
 '0_3621': {' adin',
  ' aloud',
  ' antid',
  ' eur',
  ' italy',
  ' showc',
  ' transc',
  'afort',
  'esco',
  'italy'},
 '0_3637': {'cha',
  'edited',
  'etics',
  'eting',
  'ets',
  'mexico',
  'owa',
  'stals',
  'ダ',
  '�'},
 '1_1': {' ********************************',
  ' trave',
  'atta',
  'frames',
  'gencies',
  'igate',
  'ione',
  'italy',
  'ogle',
  'yssey'},
 '1_477

In [ ]:
len(layer_idx_dict.keys())

154

In [7]:
import json 
f = open("bias_evaluation/bbq_lite/resources/Gender_identity.json")

gender_identity = json.load(f)

In [ ]:
gender_identity_set = set()
# type(gender_identity['examples'])
for example in gender_identity['examples']:
  gender_identity_set.update(example['answer_info']['ans0'])
  gender_identity_set.update(example['answer_info']['ans1'])
  gender_identity_set.update(example['answer_info']['ans2'])

In [8]:
bbq_gender = ['Barbara',
 'Betty',
 'Brian',
 'Carol',
 'Charles',
 'Christopher',
 'Daniel',
 'David',
 'Donald',
 'Donna',
 'Dorothy',
 'Edward',
 'Elizabeth',
 'George',
 'Helen',
 'James',
 'Jennifer',
 'John',
 'Joseph',
 'Karen',
 'Kenneth',
 'Linda',
 'Lisa',
 'Margaret',
 'Maria',
 'Mark',
 'Mary',
 'Michael',
 'Nancy',
 'Patricia',
 'Paul',
 'Richard',
 'Robert',
 'Ruth',
 'Sandra',
 'Sharon',
 'Steven',
 'Susan',
 'Thomas',
 'boy',
 'female',
 'girl',
 'male',
 'man',
 'woman'
]
gender_set = male_names.union(female_names).union(bbq_gender)

In [ ]:
gender_layer_idx_dict = {}
for layer in range(24):
    print(f"Scanning layer {layer}")
    for index in range(4096):
        if(index % 1000 == 0):
            print(f"Scanning index {index}")
        top_k = set([word.lower() for word, prob in wrapper.project_value_to_vocab(layer, index, 10)])
        if(len(top_k.intersection(gender_set)) >= 2):
            print(f"Layer: {layer}, Index: {index}")
            print(top_k)
            key = str(layer)+"_"+str(index)
            gender_layer_idx_dict[key] = top_k

Scanning layer 0
Scanning index 0
Layer: 0, Index: 53
{'helps', 'male', 'imir', 'stration', 'gur', 'nea', 'skip', 'ikan', 'female'}
Scanning index 1000
Scanning index 2000
Scanning index 3000
Scanning index 4000
Scanning layer 1
Scanning index 0
Scanning index 1000
Layer: 1, Index: 1040
{'mary', 'ows', 'enced', 'patrick', 'nd', 'cart', 'oman', 'allah', 'outheast', 'aq'}
Layer: 1, Index: 1393
{' mae', 'ann', ' louise', ' scotia', ' ireland', ' hungarian', ' ann', ' ilcs', 'marie', ' marie'}
Scanning index 2000
Scanning index 3000
Scanning index 4000
Scanning layer 2
Scanning index 0
Scanning index 1000
Scanning index 2000
Scanning index 3000
Scanning index 4000
Scanning layer 3
Scanning index 0
Scanning index 1000
Scanning index 2000
Layer: 3, Index: 2161
{' externaltoevaonly', 'ann', 'alg', 'antics', ' aph', 'ian', 'uran', 'antage', 'arant', 'anna'}
Scanning index 3000
Scanning index 4000
Scanning layer 4
Scanning index 0
Scanning index 1000
Scanning index 2000
Scanning index 3000
Laye

In [ ]:
temp_file = open('gender_layer_idx_dict.pkl', "wb")
pickle.dump(gender_layer_idx_dict, temp_file)


In [ ]:
glid_file = open('gender_layer_idx_dict.pkl', "rb")
glid = pickle.load(glid_file)

In [ ]:
len(glid.keys())

31

In [9]:
import json 
f = open("bias_evaluation/bbq_lite/resources/Race_ethnicity.json")

race_ethnicity = json.load(f)

In [ ]:
ethnicity_layer_idx_dict = {}
for layer in range(24):
    print(f"Scanning layer {layer}")
    for index in range(4096):
        if(index % 1000 == 0):
            print(f"Scanning index {index}")
        top_k = set([word.lower() for word, prob in wrapper.project_value_to_vocab(layer, index, 10)])
        if(len(top_k.intersection(ethnicities)) >= 1):
            print(f"Layer: {layer}, Index: {index}")
            print(top_k)
            key = str(layer)+"_"+str(index)
            ethnicity_layer_idx_dict[key] = top_k

Scanning layer 0
Scanning index 0
Scanning index 1000
Scanning index 2000
Scanning index 3000
Scanning index 4000
Scanning layer 1
Scanning index 0
Scanning index 1000
Scanning index 2000
Scanning index 3000
Layer: 1, Index: 3058
{'ariat', 'aucas', 'middle', 'west', ' faire', 'wine', 'rooms', 'asian', 'essage', 'wn'}
Layer: 1, Index: 3935
{'las', 'ulic', 'law', 'ling', 'è', 'ening', 'manac', 'netmessage', 'sol', 'hispanic'}
Scanning index 4000
Scanning layer 2
Scanning index 0
Scanning index 1000
Scanning index 2000
Scanning index 3000
Scanning index 4000
Layer: 2, Index: 4072
{'sql', 'rick', ' 0004', '️', 'lie', 'antis', 'miss', 'arab', 'cery', 'ズ'}
Scanning layer 3
Scanning index 0
Layer: 3, Index: 467
{'dispatch', 'os', 'should', 'un', 'sy', 'morning', 'df', 'arab', 'finish', 'benefit'}
Scanning index 1000
Layer: 3, Index: 1296
{'noticed', ' proble', 'risis', '��', 'atham', 'chwitz', 'ribune', 'ategic', 'asian', 'andestine'}
Layer: 3, Index: 1468
{'ickers', 'atto', 'arily', 'ided', 

In [ ]:
temp_file = open('ethnicity_layer_idx_dict.pkl', "wb")
pickle.dump(ethnicity_layer_idx_dict, temp_file)

In [10]:
ethnicity_file = open('ethnicity_layer_idx_dict.pkl', "rb")
ethnicty_dict = pickle.load(ethnicity_file)
len(ethnicty_dict.keys())

FileNotFoundError: ignored

## Intersection Base Sorting
Try to narrow down the list of vectors based on the size of their intersection

### Countries

In [40]:
import pickle
from collections import defaultdict

countries_temp_file = open("value_vector_scans/countries_layer_idx_dict.pkl", "rb")
gender_temp_file = open("value_vector_scans/gender_layer_idx_dict.pkl", "rb")
ethnicity_temp_file = open("value_vector_scans/ethnicity_layer_idx_dict.pkl", "rb")

countries_layer_idx_dict = pickle.load(countries_temp_file)
gender_layer_idx_dict = pickle.load(gender_temp_file)
ethnicity_layer_idx_dict = pickle.load(ethnicity_temp_file)


for vector_idx in countries_layer_idx_dict:
    intersection_size = len(countries_layer_idx_dict[vector_idx].intersection(set(countries)))
    countries_layer_idx_dict[vector_idx] = {
       "vector": countries_layer_idx_dict[vector_idx],
       "intersection_size": intersection_size
    }

for vector_idx in gender_layer_idx_dict:
    intersection_size = len(gender_layer_idx_dict[vector_idx].intersection(set(gender_set)))
    gender_layer_idx_dict[vector_idx] = {
       "vector": gender_layer_idx_dict[vector_idx],
       "intersection_size": intersection_size
    }
  
for vector_idx in ethnicity_layer_idx_dict:
    intersection_size = len(ethnicity_layer_idx_dict[vector_idx].intersection(set(ethnicities)))
    ethnicity_layer_idx_dict[vector_idx] = {
       "vector": ethnicity_layer_idx_dict[vector_idx],
       "intersection_size": intersection_size
    }

sorted_countries_vectors = list(map(
    lambda x: x[0],
    sorted(countries_layer_idx_dict.items(), key=lambda item: item[1]["intersection_size"], reverse=True)
    ))

sorted_gender_vectors = list(map(
    lambda x: x[0],
    sorted(gender_layer_idx_dict.items(), key=lambda item: item[1]["intersection_size"], reverse=True)
    ))

sorted_ethnicity_vectors = list(map(
    lambda x: x[0],
    sorted(ethnicity_layer_idx_dict.items(), key=lambda item: item[1]["intersection_size"], reverse=True)
    ))


def get_vector_map(sorted_vectors: list, top_k: int):
  vector_map_top_ten = {}
  for vector_location in sorted_vectors[:top_k]:
    layer, vector = vector_location.split("_")
    layer = int(layer)
    vector = int(vector)
    if(vector_map_top_ten.get(layer)):
      vector_map_top_ten[layer].append(vector)
    else:
      vector_map_top_ten[layer] = [vector]
  return vector_map_top_ten

print(get_vector_map(sorted_ethnicity_vectors,10))
print(get_vector_map(sorted_gender_vectors,10))
print(get_vector_map(sorted_countries_vectors,10))
print(get_vector_map(sorted_ethnicity_vectors,20))
print(get_vector_map(sorted_gender_vectors,20))
print(get_vector_map(sorted_countries_vectors,20))



{17: [3769], 20: [3159], 1: [3058, 3935], 2: [4072], 3: [467, 1296, 1468, 2895, 3019]}
{16: [3167], 11: [3254], 14: [3330], 0: [53], 1: [1040, 1393], 3: [2161], 4: [3111, 3680], 6: [2330]}
{1: [3458], 16: [1576, 404], 19: [53], 21: [1662], 6: [2008], 10: [3176, 3772], 12: [2117], 15: [2583]}
{17: [3769], 20: [3159], 1: [3058, 3935], 2: [4072], 3: [467, 1296, 1468, 2895, 3019, 3164], 4: [152, 3810], 5: [1299, 3845], 6: [3053], 8: [1771], 9: [968, 1127, 1908]}
{16: [3167], 11: [3254, 140, 1226], 14: [3330], 0: [53], 1: [1040, 1393], 3: [2161], 4: [3111, 3680], 6: [2330, 3121], 7: [2092], 9: [1507, 1902], 10: [3868], 12: [772, 1793], 13: [1183]}
{1: [3458], 16: [1576, 404, 2882], 19: [53, 2266, 4031], 21: [1662], 6: [2008], 10: [3176, 3772], 12: [2117], 15: [2583], 17: [2005, 2474, 2594, 2638], 23: [1159], 0: [287, 434]}
